https://www.jphwang.com/posts/nba-shot-data-analytics-visualization-with-python-pandas-and-matplotlib-part-2-grouping-data-by-area/

for hexabin tutorial


https://medium.com/@amitparikh41/college-basketball-shot-mapping-with-python-23f543528b5f

shot chart mapping

Step 1: Setting Up the Environment and Helper Functions

We'll start with helper functions to fetch and save the list of players, get team abbreviations, and categorize shots.

In [72]:
%%writefile ../src/shot_chart/nba_helpers.py

import requests
import pandas as pd
import os
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import commonallplayers
import numpy as np

def fetch_and_save_players_list():
    """Fetches the list of all players for specified seasons and saves it to a CSV file."""
    seasons = ["2023-24", "2022-23", "2021-22"]
    all_players = commonallplayers.CommonAllPlayers(is_only_current_season=0).get_data_frames()[0]

    players_data = []
    for season in seasons:
        for _, player in all_players.iterrows():
            players_data.append({
                'id': player['PERSON_ID'],
                'full_name': player['DISPLAY_FIRST_LAST'],
                'season': season
            })

    df = pd.DataFrame(players_data).drop_duplicates()
    df.to_csv('data/shot_chart_data/players_list.csv', index=False)

def load_players_list(season):
    """Loads the list of players for a specific season from a CSV file."""
    file_path = 'data/shot_chart_data/players_list.csv'
    if not os.path.exists(file_path):
        fetch_and_save_players_list()
    
    players_df = pd.read_csv(file_path)
    return players_df[players_df['season'] == season]

def get_team_abbreviation(team_name):
    """Gets the team abbreviation for a given team name."""
    team_dictionary = teams.get_teams()
    team_info = [team for team in team_dictionary if team['full_name'] == team_name]
    if not team_info:
        raise ValueError(f"No team found with name {team_name}")
    return team_info[0]['abbreviation']

def categorize_shot(row):
    """Categorizes a shot based on its location."""
    x, y = row['LOC_X'], row['LOC_Y']
    distance_from_hoop = np.sqrt(x**2 + y**2)

    if distance_from_hoop > 300:  # Over 30 ft
        return 'Backcourt', 'Beyond 30 ft'
    elif distance_from_hoop > 240:  # 24-30 ft
        if x < -80:
            return 'Deep 3 Left', '24-30 ft'
        elif x > 80:
            return 'Deep 3 Right', '24-30 ft'
        else:
            return 'Deep 3 Center', '24-30 ft'
    elif y > 237.5:
        if x < -80:
            return 'Left Corner 3', '24+ ft'
        elif x < 80:
            return 'Left Wing 3', '24+ ft'
        else:
            return 'Right Corner 3', '24+ ft'
    elif y > 142.5:
        if x < -80:
            return 'Left Wing 3', '24+ ft'
        elif x < 0:
            return 'Left Top of Key 3', '20-24 ft'
        else:
            return 'Right Top of Key 3', '20-24 ft'
    elif y > 47.5:
        if x < -80:
            return 'Left Baseline Mid-range', '10-20 ft'
        elif x < -10:
            return 'Left Elbow Mid-range', '10-20 ft'
        elif x < 10:
            return 'Center Mid-range', '10-20 ft'
        elif x < 80:
            return 'Right Elbow Mid-range', '10-20 ft'
        else:
            return 'Right Baseline Mid-range', '10-20 ft'
    elif y > 0:
        if x < -80:
            return 'Left of Near Basket', '0-10 ft'
        elif x < 80:
            return 'Center of Near Basket', '0-10 ft'
        else:
            return 'Right of Near Basket', '0-10 ft'
    else:
        return 'Unknown', 'Unknown'


Overwriting ../src/shot_chart/nba_helpers.py


Step 2: Fetching Shots Data

Next, we'll write functions to fetch shots data for both offensive and defensive sides.

In [73]:
%%writefile ../src/shot_chart/nba_shots.py

import pandas as pd
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams
from shot_chart.nba_helpers import get_team_abbreviation

def fetch_shots_data(name, is_team, season, opponent_team=None, opponent_player=None, game_date=None):
    """Fetches shots data for a team or player for a given season with optional filters."""
    if is_team:
        team_dictionary = teams.get_teams()
        team_info = [team for team in team_dictionary if team['full_name'] == name]
        
        if not team_info:
            raise ValueError(f"No team found with name {name}")
        
        team_id = team_info[0]['id']
        print(f"Fetching data for Team ID: {team_id}")
        
        shotchart = shotchartdetail.ShotChartDetail(
            team_id=team_id,
            player_id=0,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star=['Regular Season', 'Playoffs']
        )
    else:
        player_dictionary = players.get_players()
        player_info = [player for player in player_dictionary if player['full_name'] == name]
        
        if not player_info:
            raise ValueError(f"No player found with name {name}")
        
        player_id = player_info[0]['id']
        print(f"Fetching data for Player ID: {player_id}")
        
        shotchart = shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=player_id,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star=['Regular Season', 'Playoffs']
        )
    
    data = shotchart.get_data_frames()[0]

    if opponent_team:
        opponent_abbreviation = get_team_abbreviation(opponent_team)
        data = data[(data['HTM'] == opponent_abbreviation) | (data['VTM'] == opponent_abbreviation)]
    
    if opponent_player:
        opponent_dictionary = players.get_players()
        opponent_info = [player for player in opponent_dictionary if player['full_name'] == opponent_player]
        if opponent_info:
            opponent_player_id = opponent_info[0]['id']
            data = data[data['PLAYER_ID'] == opponent_player_id]

    if game_date:
        data = data[data['GAME_DATE'] == game_date.replace('-', '')]

    return data

def fetch_defensive_shots_data(name, is_team, season, opponent_team=None, opponent_player=None, game_date=None):
    """Fetches defensive shots data for a team or player for a given season with optional filters."""
    if is_team:
        team_abbr = get_team_abbreviation(name)
        shotchart = shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=0,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star=['Regular Season', 'Playoffs']
        )
        
        data = shotchart.get_data_frames()[0]
        defensive_shots = data[(data['HTM'] == team_abbr) | (data['VTM'] == team_abbr)]
        defensive_shots = defensive_shots[defensive_shots['TEAM_NAME'] != name]

    else:
        player_dictionary = players.get_players()
        player_info = [player for player in player_dictionary if player['full_name'] == name]
        
        if not player_info:
            raise ValueError(f"No player found with name {name}")
        
        player_id = player_info[0]['id']
        print(f"Fetching data for Player ID: {player_id}")
        
        shotchart = shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=0,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star=['Regular Season', 'Playoffs']
        )
        
        data = shotchart.get_data_frames()[0]
        defensive_shots = data[data['PLAYER_ID'] == player_id]

    if opponent_team:
        opponent_abbreviation = get_team_abbreviation(opponent_team)
        defensive_shots = defensive_shots[(defensive_shots['HTM'] == opponent_abbreviation) | (defensive_shots['VTM'] == opponent_abbreviation)]

    if opponent_player:
        opponent_dictionary = players.get_players()
        opponent_info = [player for player in opponent_dictionary if player['full_name'] == opponent_player]
        if opponent_info:
            opponent_player_id = opponent_info[0]['id']
            defensive_shots = defensive_shots[defensive_shots['PLAYER_ID'] == opponent_player_id]

    if game_date:
        defensive_shots = defensive_shots[defensive_shots['GAME_DATE'] == game_date.replace('-', '')]

    return defensive_shots


Overwriting ../src/shot_chart/nba_shots.py


Step 3: Plotting Functions

We'll create functions to plot the court and shot charts.

In [74]:
%%writefile ../src/shot_chart/nba_plotting.py

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_court(ax=None):
    """Plots the basketball court on a given axis."""
    if ax is None:
        ax = plt.gca()
    
    ax.set_xlim(-250, 250)
    ax.set_ylim(-47.5, 422.5)
    ax.set_aspect('equal')
    
    court_elements = [
        plt.Circle((0, 0), radius=7.5, linewidth=2, color='black', fill=False),  # Hoop
        plt.Rectangle((-30, -10), 60, -1, linewidth=2, color='black'),  # Backboard
        plt.Rectangle((-80, -47.5), 160, 190, linewidth=2, color='black', fill=False),  # Paint
        plt.Circle((0, 142.5), radius=60, linewidth=2, color='black', fill=False),  # Free throw top arc
        plt.Circle((0, 142.5), radius=60, linewidth=2, color='black', fill=False, linestyle='dashed'),  # Free throw bottom arc
        patches.Arc((0, 0), 475, 475, theta1=0, theta2=180, linewidth=2, color='black'),  # 3-point arc
        plt.Rectangle((-250, -47.5), 500, 470, linewidth=2, color='black', fill=False),  # Outer lines
    ]
    
    for element in court_elements:
        ax.add_patch(element)
    
    return ax

def plot_shot_chart_hexbin(shots, title, opponent, court_color='white'):
    """Plots a hexbin shot chart."""
    plt.figure(figsize=(12, 11))
    ax = plt.gca()
    ax.set_facecolor(court_color)
    plot_court(ax)
    
    hexbin = plt.hexbin(
        shots['LOC_X'], shots['LOC_Y'], C=shots['SHOT_MADE_FLAG'], 
        gridsize=40, extent=(-250, 250, -47.5, 422.5), cmap='Blues', edgecolors='grey'
    )
    
    cb = plt.colorbar(hexbin, ax=ax, orientation='vertical')
    cb.set_label('Shooting Percentage')
    
    total_attempts = len(shots)
    total_made = shots['SHOT_MADE_FLAG'].sum()
    overall_percentage = total_made / total_attempts if total_attempts > 0 else 0
    
    opponent_text = f" against {opponent}" if opponent else " against the rest of the league"
    
    plt.text(0, 450, f"Total Shots: {total_attempts}", fontsize=12, ha='center')
    plt.text(0, 430, f"Total Made: {total_made}", fontsize=12, ha='center')
    plt.text(0, 410, f"Overall Percentage: {overall_percentage:.2%}", fontsize=12, ha='center')
    
    plt.title(f"{title}{opponent_text}", pad=50)  # Adjusted title to include opponent
    plt.xlim(-250, 250)
    plt.ylim(-47.5, 422.5)
    
    plt.tight_layout()  # Ensures that the plot elements don't overlap
    return plt.gcf()



Overwriting ../src/shot_chart/nba_plotting.py


Step 4: Efficiency Calculation Functions

We'll write functions to calculate shot efficiency and team fit.

In [75]:
%%writefile ../src/shot_chart/nba_efficiency.py

import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import os
from shot_chart.nba_helpers import categorize_shot
from shot_chart.nba_shots import fetch_shots_data, fetch_defensive_shots_data

def calculate_efficiency(shots):
    """Calculates the efficiency of shots."""
    shots['Area'], shots['Distance'] = zip(*shots.apply(categorize_shot, axis=1))
    summary = shots.groupby(['Area', 'Distance']).agg(
        Attempts=('SHOT_MADE_FLAG', 'size'),
        Made=('SHOT_MADE_FLAG', 'sum')
    ).reset_index()
    summary['Efficiency'] = summary['Made'] / summary['Attempts']
    return summary

def calculate_team_fit(home_efficiency, opponent_efficiency):
    """Calculates the team fit using MAE and MAPE."""
    common_areas = set(home_efficiency['Area']).intersection(set(opponent_efficiency['Area']))
    
    home_efficiency_common = home_efficiency[home_efficiency['Area'].isin(common_areas)]
    opponent_efficiency_common = opponent_efficiency[opponent_efficiency['Area'].isin(common_areas)]
    
    mae = mean_absolute_error(home_efficiency_common['Efficiency'], opponent_efficiency_common['Efficiency'])
    mape = mean_absolute_percentage_error(home_efficiency_common['Efficiency'], opponent_efficiency_common['Efficiency'])
    
    return mae, mape

def create_mae_table(home_team, season, all_teams):
    """Creates a table of MAE values for the given home team against all opponents."""
    mae_list = []
    home_shots = fetch_shots_data(home_team, True, season)
    home_efficiency = calculate_efficiency(home_shots)

    for opponent in all_teams:
        if opponent == home_team:
            continue
        
        # Ensure the season is passed to fetch_defensive_shots_data
        opponent_shots = fetch_defensive_shots_data(opponent, True, season)
        opponent_efficiency = calculate_efficiency(opponent_shots)
        
        mae, mape = calculate_team_fit(home_efficiency, opponent_efficiency)
        
        mae_list.append({
            'Home Team': home_team,
            'Opponent Team': opponent,
            'MAE': mae,
            'MAPE': mape,
            'Season': season
        })
    
    mae_df = pd.DataFrame(mae_list)
    mae_df = mae_df.sort_values(by='MAE')
    return mae_df

def save_mae_table(mae_df, file_path):
    """Saves the MAE table to a CSV file."""
    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)
        mae_df = pd.concat([existing_df, mae_df]).drop_duplicates()
    mae_df.to_csv(file_path, index=False)

def load_mae_table(file_path):
    """Loads the MAE table from a CSV file if it exists."""
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    return None

def get_seasons_range(mae_df):
    """Returns the minimum and maximum seasons in the MAE DataFrame."""
    min_season = mae_df['Season'].min()
    max_season = mae_df['Season'].max()
    return min_season, max_season




Overwriting ../src/shot_chart/nba_efficiency.py


Step 5: Main Function

Finally, we'll create the main function that ties everything together and performs the analysis.

In [76]:
%%writefile ../src/shot_chart/shot_chart_main.py

import os
import pandas as pd
import matplotlib.pyplot as plt
from nba_api.stats.static import players, teams
from shot_chart.nba_helpers import get_team_abbreviation, categorize_shot
from shot_chart.nba_shots import fetch_shots_data, fetch_defensive_shots_data
from shot_chart.nba_plotting import plot_shot_chart_hexbin
from shot_chart.nba_efficiency import calculate_efficiency, create_mae_table, save_mae_table, load_mae_table, get_seasons_range

def preload_mae_tables(entity_name, season):
    """Preload MAE tables for all teams to speed up future calculations."""
    mae_df_all_path = f'data/shot_chart_data/{entity_name}_mae_table_all_{season}.csv'
    mae_df_all = load_mae_table(mae_df_all_path)
    if mae_df_all is None:
        mae_df_all = create_mae_table(entity_name, season, [team['full_name'] for team in teams.get_teams()])
        save_mae_table(mae_df_all, mae_df_all_path)
    return mae_df_all

def create_and_save_mae_table_specific(entity_name, season, opponent_name):
    """Create and save the MAE table for a specific opponent."""
    mae_df_specific_path = f'data/shot_chart_data/{entity_name}_mae_table_specific_{season}.csv'
    mae_df_specific = load_mae_table(mae_df_specific_path)
    
    print(f"Debug: Opponent name before calling create_mae_table: {opponent_name}")
    
    if mae_df_specific is None or opponent_name not in mae_df_specific['Opponent Team'].values:
        mae_df_specific = create_mae_table(entity_name, season, [opponent_name])
        
        print(f"Debug: MAE table head after creation for opponent {opponent_name}:\n{mae_df_specific.head()}")
        
        save_mae_table(mae_df_specific, mae_df_specific_path)
    
    return mae_df_specific

def create_and_save_mae_table_all(entity_name, season):
    """Load the preloaded MAE table for all teams."""
    mae_df_all_path = f'data/shot_chart_data/{entity_name}_mae_table_all_{season}.csv'
    mae_df_all = load_mae_table(mae_df_all_path)
    if mae_df_all is None:
        mae_df_all = preload_mae_tables(entity_name, season)
    return mae_df_all

def main():
    # Step 1: Select analysis type (offensive, defensive, or both)
    analysis_type = input("Select analysis type (offensive/defensive/both): ").strip().lower()
    if analysis_type not in ['offensive', 'defensive', 'both']:
        print("Invalid selection. Please enter 'offensive', 'defensive', or 'both'.")
        return
    
    entity_type = input("Analyze a Team or Player? (team/player): ").strip().lower()
    if entity_type not in ['team', 'player']:
        print("Invalid selection. Please enter 'team' or 'player'.")
        return
    
    entity_name = input(f"Enter the {entity_type} name: ").strip()
    season = input("Enter the season (e.g., 2023-24): ").strip()
    
    opponent_type = input("Compare against all teams or a specific team? (all/specific): ").strip().lower()
    if opponent_type not in ['all', 'specific']:
        print("Invalid selection. Please enter 'all' or 'specific'.")
        return
    
    opponent_name = None
    if opponent_type == 'specific':
        opponent_name = input("Enter the opponent team name: ").strip()
    
    # Preload MAE tables for all teams
    mae_df_all = preload_mae_tables(entity_name, season)
    
    # Fetch and display offensive data
    shots = fetch_shots_data(entity_name, entity_type == 'team', season, opponent_name)
    print(shots.head())
    
    efficiency = calculate_efficiency(shots)
    print(f"Offensive Efficiency for {entity_name}:")
    print(efficiency)
    
    # Update the plot call to include the opponent's name or indicate it's against all teams
    fig = plot_shot_chart_hexbin(shots, f'{entity_name} Shot Chart', opponent=opponent_name if opponent_name else "the rest of the league")
    plt.show()
    
    if opponent_type == 'specific':
        # MAE calculation and saving for specific team
        mae_df_specific = create_and_save_mae_table_specific(entity_name, season, opponent_name)
        print(f"MAE Table for {entity_name} against {opponent_name}:")
        print(mae_df_specific)
    else:
        # MAE calculation and loading for all teams
        print(f"MAE Table for {entity_name} against all teams:")
        print(mae_df_all)
    
    min_season, max_season = get_seasons_range(mae_df_all)
    print(f"MAE Table available for seasons from {min_season} to {max_season}.")

    # If the analysis type is "both", also perform defensive analysis here
    if analysis_type == 'both':
        # Fetch and display defensive data for the specified team
        defensive_shots = fetch_defensive_shots_data(entity_name, True, season, opponent_name)
        defensive_efficiency = calculate_efficiency(defensive_shots)
        print(f"Defensive Efficiency for {entity_name}:")
        print(defensive_efficiency)
        
        # Update the plot call to include the opponent's name or indicate it's against all teams
        fig = plot_shot_chart_hexbin(defensive_shots, f'{entity_name} Defensive Shot Chart', opponent=opponent_name if opponent_name else "the rest of the league")
        plt.show()
        
        if opponent_type == 'specific':
            # MAE calculation for defensive analysis against the specific opponent
            mae_df_specific = create_and_save_mae_table_specific(entity_name, season, opponent_name)
            print(f"Defensive MAE Table for {entity_name} against {opponent_name}:")
            print(mae_df_specific)

if __name__ == "__main__":
    main()



Overwriting ../src/shot_chart/shot_chart_main.py


In [77]:
%%writefile ../src/shot_chart_streamlit_app.py

import os
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
from nba_api.stats.static import players, teams
from shot_chart.nba_helpers import get_team_abbreviation, categorize_shot
from shot_chart.nba_shots import fetch_shots_data, fetch_defensive_shots_data
from shot_chart.nba_plotting import plot_shot_chart_hexbin
from shot_chart.nba_efficiency import calculate_efficiency, create_mae_table, save_mae_table, load_mae_table, get_seasons_range

@st.cache_data
def preload_mae_tables(entity_name, season):
    """Preload MAE tables for all teams to speed up future calculations."""
    mae_df_all_path = f'data/shot_chart_data/{entity_name}_mae_table_all_{season}.csv'
    mae_df_all = load_mae_table(mae_df_all_path)
    if mae_df_all is None:
        mae_df_all = create_mae_table(entity_name, season, [team['full_name'] for team in teams.get_teams()])
        save_mae_table(mae_df_all, mae_df_all_path)
    return mae_df_all

@st.cache_data
def create_and_save_mae_table_specific(entity_name, season, opponent_name):
    """Create and save the MAE table for a specific opponent."""
    mae_df_specific_path = f'data/shot_chart_data/{entity_name}_mae_table_specific_{season}.csv'
    mae_df_specific = load_mae_table(mae_df_specific_path)
    
    if mae_df_specific is None or opponent_name not in mae_df_specific['Opponent Team'].values:
        mae_df_specific = create_mae_table(entity_name, season, [opponent_name])
        save_mae_table(mae_df_specific, mae_df_specific_path)
    
    return mae_df_specific

@st.cache_data
def create_and_save_mae_table_all(entity_name, season):
    """Load the preloaded MAE table for all teams."""
    mae_df_all_path = f'data/shot_chart_data/{entity_name}_mae_table_all_{season}.csv'
    mae_df_all = load_mae_table(mae_df_all_path)
    if mae_df_all is None:
        mae_df_all = preload_mae_tables(entity_name, season)
    return mae_df_all

@st.cache_data
def get_teams_list():
    """Get the list of NBA teams."""
    return [team['full_name'] for team in teams.get_teams()]

@st.cache_data
def get_players_list():
    """Get the list of NBA players."""
    return [player['full_name'] for player in players.get_players()]

def main():
    st.title("NBA Shot Analysis")
    
    analysis_type = st.selectbox("Select analysis type", options=["offensive", "defensive", "both"])
    
    entity_type = st.selectbox("Analyze a Team or Player?", options=["team", "player"])
    
    if entity_type == "team":
        entity_name = st.selectbox("Select a Team", options=get_teams_list())
    else:
        entity_name = st.selectbox("Select a Player", options=get_players_list())
    
    season = st.selectbox("Select the season", options=["2023-24", "2022-23", "2021-22", "2020-21"])
    
    opponent_type = st.selectbox("Compare against all teams or a specific team?", options=["all", "specific"])
    
    opponent_name = None
    if opponent_type == "specific":
        opponent_name = st.selectbox("Select an Opponent Team", options=get_teams_list())
    
    if st.button("Run Analysis"):
        # Preload MAE tables for all teams
        mae_df_all = preload_mae_tables(entity_name, season)
        
        # Fetch and display offensive data
        shots = fetch_shots_data(entity_name, entity_type == 'team', season, opponent_name)
        st.write("Shot Data")
        st.dataframe(shots.head())
        
        efficiency = calculate_efficiency(shots)
        st.write(f"Offensive Efficiency for {entity_name}:")
        st.dataframe(efficiency)
        
        # Plot shot chart
        fig = plot_shot_chart_hexbin(shots, f'{entity_name} Shot Chart', opponent=opponent_name if opponent_name else "the rest of the league")
        st.pyplot(fig)
        
        if opponent_type == 'specific':
            # MAE calculation and saving for specific team
            mae_df_specific = create_and_save_mae_table_specific(entity_name, season, opponent_name)
            st.write(f"MAE Table for {entity_name} against {opponent_name}:")
            st.dataframe(mae_df_specific)
        else:
            # MAE calculation and loading for all teams
            st.write(f"MAE Table for {entity_name} against all teams:")
            st.dataframe(mae_df_all)
        
        min_season, max_season = get_seasons_range(mae_df_all)
        st.write(f"MAE Table available for seasons from {min_season} to {max_season}.")
    
        # If the analysis type is "both", also perform defensive analysis here
        if analysis_type == 'both':
            # Fetch and display defensive data for the specified team
            defensive_shots = fetch_defensive_shots_data(entity_name, True, season, opponent_name)
            defensive_efficiency = calculate_efficiency(defensive_shots)
            st.write(f"Defensive Efficiency for {entity_name}:")
            st.dataframe(defensive_efficiency)
            
            # Plot defensive shot chart
            fig = plot_shot_chart_hexbin(defensive_shots, f'{entity_name} Defensive Shot Chart', opponent=opponent_name if opponent_name else "the rest of the league")
            st.pyplot(fig)
            
            if opponent_type == 'specific':
                # MAE calculation for defensive analysis against the specific opponent
                mae_df_specific = create_and_save_mae_table_specific(entity_name, season, opponent_name)
                st.write(f"Defensive MAE Table for {entity_name} against {opponent_name}:")
                st.dataframe(mae_df_specific)

if __name__ == "__main__":
    main()


Overwriting ../src/shot_chart_streamlit_app.py
